# PPO - 2-way Single Intersection

## Imports

In [1]:
import os

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from sumo_rl import SumoEnvironment

from tyre_pm_reward import tyre_pm_reward

c:\Users\Justin Mak\anaconda3\envs\rl\lib\site-packages\gymnasium\envs\registration.py:693: UserWarning: WARN: Overriding environment GymV26Environment-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


## Create Model and Instantiate Agent

In [2]:
env = SumoEnvironment(
    net_file="nets/2way-single-intersection/single-intersection.net.xml",
    route_file="nets/2way-single-intersection/single-intersection-vhvh.rou.xml",
    out_csv_name="outputs/2way-single-intersection/ppo",
    single_agent=True,
    use_gui=False,
    num_seconds=1e5,
    reward_fn=tyre_pm_reward
)
env = Monitor(env)  # wrap env to know episode reward, length, time

model = PPO(
    "MlpPolicy",
    env,
    learning_rate=5e-3,
    n_steps=256,
    batch_size=256,
    n_epochs=5,
    gamma=0.95,
    gae_lambda=0.99,
    clip_range=0.3,
    ent_coef=0.09,
    vf_coef=0.05,
    max_grad_norm=0.9,
    tensorboard_log="./logs/2way-single-intersection/ppo_test",
    verbose=1
)

Using cuda device
Wrapping the env in a DummyVecEnv.


## Train and Evaluate Agent

In [3]:
model.learn(total_timesteps=1e5, progress_bar=True)

Logging to ./logs/2way-single-intersection/ppo_test\PPO_1


Output()

----------------------------
| time/              |     |
|    fps             | 33  |
|    iterations      | 1   |
|    time_elapsed    | 7   |
|    total_timesteps | 256 |
----------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 36          |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 512         |
| train/                  |             |
|    approx_kl            | 0.031264618 |
|    clip_fraction        | 0.0852      |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.38       |
|    explained_variance   | -0.00132    |
|    learning_rate        | 0.005       |
|    loss                 | 1.23e+03    |
|    n_updates            | 5           |
|    policy_gradient_loss | -0.0185     |
|    value_loss           | 2.5e+04     |
-----------------------------------------


-----------------------------------------
| time/                   |             |
|    fps                  | 38          |
|    iterations           | 3           |
|    time_elapsed         | 19          |
|    total_timesteps      | 768         |
| train/                  |             |
|    approx_kl            | 0.008830533 |
|    clip_fraction        | 0.00234     |
|    clip_range           | 0.3         |
|    entropy_loss         | -1.34       |
|    explained_variance   | -0.00102    |
|    learning_rate        | 0.005       |
|    loss                 | 600         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0121     |
|    value_loss           | 1.23e+04    |
-----------------------------------------


...

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2e+04     |
|    ep_rew_mean          | -8.65e+03 |
| time/                   |           |
|    fps                  | 12        |
|    iterations           | 387       |
|    time_elapsed         | 8122      |
|    total_timesteps      | 99072     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.3       |
|    entropy_loss         | -3.7e-08  |
|    explained_variance   | -1.19e-07 |
|    learning_rate        | 0.005     |
|    loss                 | 0.0323    |
|    n_updates            | 1930      |
|    policy_gradient_loss | -9.31e-10 |
|    value_loss           | 0.634     |
---------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2e+04     |
|    ep_rew_mean          | -8.65e+03 |
| time/                   |           |
|    fps                  | 12        |
|    iterations           | 388       |
|    time_elapsed         | 8154      |
|    total_timesteps      | 99328     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.3       |
|    entropy_loss         | -3.7e-08  |
|    explained_variance   | 2.38e-07  |
|    learning_rate        | 0.005     |
|    loss                 | 0.0231    |
|    n_updates            | 1935      |
|    policy_gradient_loss | 2.09e-08  |
|    value_loss           | 0.525     |
---------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2e+04     |
|    ep_rew_mean          | -8.65e+03 |
| time/                   |           |
|    fps                  | 12        |
|    iterations           | 389       |
|    time_elapsed         | 8186      |
|    total_timesteps      | 99584     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.3       |
|    entropy_loss         | -3.7e-08  |
|    explained_variance   | 3.58e-07  |
|    learning_rate        | 0.005     |
|    loss                 | 0.0249    |
|    n_updates            | 1940      |
|    policy_gradient_loss | 2.98e-09  |
|    value_loss           | 0.555     |
---------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2e+04     |
|    ep_rew_mean          | -8.65e+03 |
| time/                   |           |
|    fps                  | 12        |
|    iterations           | 390       |
|    time_elapsed         | 8217      |
|    total_timesteps      | 99840     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.3       |
|    entropy_loss         | -3.7e-08  |
|    explained_variance   | 8.34e-07  |
|    learning_rate        | 0.005     |
|    loss                 | 0.0182    |
|    n_updates            | 1945      |
|    policy_gradient_loss | 6.15e-09  |
|    value_loss           | 0.332     |
---------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 2e+04     |
|    ep_rew_mean          | -7.81e+03 |
| time/                   |           |
|    fps                  | 12        |
|    iterations           | 391       |
|    time_elapsed         | 8241      |
|    total_timesteps      | 100096    |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.3       |
|    entropy_loss         | -3.7e-08  |
|    explained_variance   | 1.37e-06  |
|    learning_rate        | 0.005     |
|    loss                 | 0.0126    |
|    n_updates            | 1950      |
|    policy_gradient_loss | 1.12e-09  |
|    value_loss           | 0.306     |
---------------------------------------


In [4]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=1)
print(mean_reward)
print(std_reward)

-4477.13758
0.0


## Save Model

In [5]:
model.save(os.path.join("saved_models","ppo_single_intersection"))